In [1]:
import mysql
from mysql.connector import MySQLConnection, Error
from python_mysql_dbconfig import read_db_config # Read database configuration file and return a dictionary database object

In [2]:
def create_msql_database(name_database):
    """Function create MYSQL database to save into db config file"""
    mydb = mysql.connector.connect(host='localhost',
                                      user='root',
                                      password='root')
    
    mycursor = mydb.cursor()
    mycursor.execute("SHOW DATABASES")
    databases = [x[0] for x in mycursor]
    if name_database not in databases:
        mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {name_database}")
    # Check database

    
def create_msql_table(query_create):
    """
    Execute query to create table
    :query_create:: to create table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')
        cursor = conn.cursor()
        print('Creating Mysql table...')
        cursor.execute(query_create)
        print('Table created')
    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()

def insert_msql_table(data_dir):
    """execute query to insert values to table
    :query_table:: to create scheme table
    :table_values:: to be inserted table
    """

    conn = None
    try:
        db_config = read_db_config() #Using configuration file to connect to database
        print('Connecting to MYSQL database...')
        conn = MySQLConnection(**db_config)
        print('Connected to MYSQL database')

        cursor = conn.cursor()
        print('creating MYSQL table...')
        cursor.execute(data_dir)

        conn.commit()
        print('MYSQL table created')

    except Error as e:
        print("Error: ", e)

    finally:
        cursor.close()
        conn.close()



In [3]:

query_create = "CREATE TABLE Logs (Id INTEGER PRIMARY KEY, Num VARCHAR(30));"
# Create the table (if it doesn't exist)
create_network_table = "CREATE TABLE IF NOT EXIST network (net_id VARCHAR(255) UNIQUE, \
                                                            net_name VARCHAR(255), cidr VARCHAR(255) UNIQUE, \
                                                                net_range VARCHAR(255), net_type VARCHAR(255), \
                                                                    asn VARCHAR(255), org_cust_id VARCHAR(255), updated DATE);"

In [ ]:
########

import_tsv_to_mysql(
    host="localhost",
    database="your_database",
    user="your_user",
    password="your_password",
    tsv_file_path="/path/to/your/data.tsv",
    table_name="people",
    skip_header=True
)